<a href="https://colab.research.google.com/github/boothmanrylan/sequencer/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports and installs

In [ ]:
# standard python imports
import os

In [ ]:
# earth engine api import and authentication
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
# pip installable modules

try:
    import geemap.foliumap as geemap
except:
    !pip install geemap
    import geemap.foliumap as geemap

In [ ]:
# autoreload all python modules before executing code block
%load_ext autoreload
%autoreload 2

In [ ]:
# install and import my packages from github
from google.colab import drive, auth

drive.mount('/content/drive/')
%cd /content/drive/My\ Drive/colaboratory

# auth.authenticate_user()

# install msslib
! if [ ! -d ./msslib ]; then git clone https://github.com/boothmanrylan/msslib.git; fi
%cd msslib
! git pull
try:
    from msslib import msslib
except:
    ! pip install .
    from msslib import msslib
%cd ../

# install sequencer
! if [ ! -d ./sequencer ]; then git clone https://github.com/boothmanrylan/sequencer.git; fi
%cd sequencer
! git pull
try:
    import sequencer
except:
    ! pip install .
    import sequencer
%cd ../

# install MSSForestDisturbances
! if [ ! -d ./MSSForestDisturbances ]; then git clone https://github.com/boothmanrylan/MSSForestDisturbances.git; fi
%cd MSSForestDisturbances
! git pull
try:
    import mfd
except:
    ! pip install .
    import mfd
%cd ../

# Reload

Run these blocks if the modules have been updated since the runtime was started

In [ ]:
# reload msslib
%cd ./msslib
! git pull
%cd ../

In [ ]:
# reload sequencer
%cd ./sequencer/
! git pull
%cd ../

In [ ]:
# reload MSSForestDisturbances
%cd ./MSSForestDisturbances/
! git pull
%cd ../

# Test

In [ ]:
def match_sequence(events, sequence, offset=0):
    events = ee.Image(events)
    sequence = ee.List(sequence)
    offset = ee.Number(offset)

    num_events = events.bandNames().length()
    seq_length = sequence.length()

    seq_im = ee.Image.constant(sequence)

    indices = ee.List.sequence(0, num_events.subtract(seq_length))
    indices_im = ee.Image.constant(indices)

    def _check(index, prev):
        prev = ee.Image(prev)
        compare_to = events.slice(index, seq_length.add(index))
        result = seq_im.eq(compare_to).reduce(ee.Reducer.allNonZero())
        return prev.addBands(result.selfMask())

    checks = ee.Image(indices.iterate(_check, ee.Image())).slice(1)
    matches = checks.multiply(indices_im).add(offset)
    return matches.reduce(ee.Reducer.min())

In [ ]:
quebec_fires = ee.ImageCollection("users/boothmanrylan/MSSForestDisturbances/test2")
aoi = ee.Geometry.Point([-76.90250107516832, 51.526388298517745])

events = quebec_fires.filterBounds(aoi)
events = events.map(mfd.postprocess.postprocess)
events = events.map(mfd.postprocess.squash_extra_classes)
carried_forward_events = mfd.postprocess.carry_observations_forward(events)

burns = match_sequence(carried_forward_events, ee.List([1, 1, 1, 1]), 0)
burn_years = mfd.postprocess.get_year_from_index(events, burns)


In [ ]:
Map = geemap.Map()
Map.addLayer(burns, {'min': 0, 'max': 1}, 'Burns')
Map.addLayer(burn_years, {'min': 1972, 'max': 1984}, 'Dated Burns')
Map.centerObject(aoi, 10)
Map